In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import librosa
import librosa.display
import IPython.display as ipd

from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

In [2]:
def calculate_mfccs(waveform, samplingrate, number_coeffs=128):
    '''
    Caluculate Mel frequency cepstral coefficients.
    Input:
    waveform: array representing the waveform originating from audio file
    samplingrate: sampling rate from audio
    number_coeffs: total number of coefficients to calculate, default is 20
    Output: array of mfccs
    '''
    mfccs = librosa.feature.mfcc(y=waveform, sr=samplingrate,
                                 n_mfcc=number_coeffs)
    return mfccs


def calculate_specbandwidth(waveform, samplingrate, order_p=2):
    '''
    Calculates spectral bandwithd of oder p
    '''
    specbw = librosa.feature.spectral_bandwidth(y=waveform, sr=samplingrate,
                                                p=order_p)

    return specbw


def calculate_spectralcentroid(waveform, samplingrate, nfft=2048, hoplength=512):
    '''
    Calculate spectral centroid
    '''
    specent = librosa.feature.spectral_centroid(y=waveform,
                                                sr=samplingrate, n_fft=nfft,
                                                hop_length=hoplength)

    return specent


def calculate_chroma(waveform, samplingrate, nfft=2048, hoplength=512):
    '''
    Chroma Feature calculation with a hop length of 512 as default

    '''
    chromafeat = librosa.feature.chroma_stft(y=waveform, sr=samplingrate,
                                             n_fft=nfft,
                                             hop_length=hoplength)

    return chromafeat


def calculate_zerocrossrate(waveform, framelength=2048, hoplength=512,):
    '''
    Caluculate Zero crossing rate.

    '''
    zerocr = librosa.feature.zero_crossing_rate(y=waveform, frame_length=framelength,
                                                hop_length=hoplength)

    return zerocr


def calculate_spectralroff(waveform, samplingrate, nfft=2048, hoplength=512):
    '''
    Calculate spectral roll off of the waveform
    '''
    specroff = librosa.feature.spectral_rolloff(y=waveform,
                                                sr=samplingrate, n_fft=nfft,
                                                hop_length=hoplength)

    return specroff

def normalize(array):
    ''' 
    simple function to normalize an array
    requires an array as input, returns a normalized array
    '''
    return array / np.linalg.norm(array)

In [3]:
 # x, sr = reading_audiofile("example3.wav")

In [4]:
def get_add_features(x, sr):
    '''
    Extracts all additional features of a given sample (x: nd-array 1-dim , sr: sample rate)
    returns following arrays: mfcc, specbandwith, spectralcentroid, chroma, zeorcrossrate, spectralroff
    '''
    mfcc = calculate_mfccs(x, sr, number_coeffs=20)   # changed number_coeffs to 20  -> #shape (20,70)
    specbandwith = calculate_specbandwidth(x, sr, order_p=2)  # shape(1,70)
    spectralcentroid = calculate_spectralcentroid(x, sr, nfft=2048, hoplength=512)  #(shape1,70)
    chroma = calculate_chroma(x, sr, nfft=2048, hoplength=512) # shape (12,70)
    zerocrossrate = calculate_zerocrossrate(x, framelength=2048, hoplength=512,) #shape (1,70)
    spectralroff = calculate_spectralroff(x, sr, nfft=2048, hoplength=512) #shape (1,70)
    
    return mfcc, specbandwith, spectralcentroid, chroma, zerocrossrate, spectralroff


def extracting_add_features(list_from_db, desired_length=6, pause=1, num_steps=3):
    '''
    Extracts all additional features (with gett_add_features function) of a list (provided by list_from_db function)
    Requires list, desired_length in sec (default 6 sec), pause in sec (for repeat-padding, default 1 sec)
    
    returns a dictionary with keys: mfcc, specbandwith, spectralcentroid, chroma, zeorcrossrate, spectralroff, style, bpm
                        each contains a list (with features of all samples from list_from_db)
    '''
    
    
    feat_dict = {'mfcc':[], 'specbandwith':[], 'spectralcentroid':[], 'chroma':[],
                         'zerocrossrate':[], 'spectralroff':[], 'style':[], 'bpm':[]}
    for audio_sample in list_from_db:
        filename = path + audio_sample[1]
        length = audio_sample[0]
        style = audio_sample[2]
        bpm = audio_sample[3]
        
        if length < desired_length: 
            x, sr = reading_audiofile(filename, modus_operandi="repeat",
                              desired_length=desired_length, offset=0.0, pause=pause)
            mfcc, specbandwith, spectralcentroid, chroma, zerocrossrate, spectralroff = get_add_features(x, sr)
            feat_dict['mfcc'].append(mfcc)
            feat_dict['specbandwith'].append(specbandwith)
            feat_dict['spectralcentroid'].append(spectralcentroid)
            feat_dict['chroma'].append(chroma)
            feat_dict['zerocrossrate'].append(zerocrossrate)
            feat_dict['spectralroff'].append(spectralroff)
            feat_dict['style'].append(style)
            feat_dict['bpm'].append(bpm)
            
                      
        if length > desired_length:
            steps=int(length//desired_length)
            if steps > (num_steps -1):
                steps = (num_steps -1)
            for i in range(steps):
                x, sr = reading_audiofile(filename, modus_operandi="no_change",
                            desired_length=desired_length, offset=i*desired_length)
                mfcc, specbandwith, spectralcentroid, chroma, zerocrossrate, spectralroff = get_add_features(x, sr)
                feat_dict['mfcc'].append(mfcc)
                feat_dict['specbandwith'].append(specbandwith)
                feat_dict['spectralcentroid'].append(spectralcentroid)
                feat_dict['chroma'].append(chroma)
                feat_dict['zerocrossrate'].append(zerocrossrate)
                feat_dict['spectralroff'].append(spectralroff) 
                feat_dict['style'].append(style)
                feat_dict['bpm'].append(bpm)
                
            x, sr = reading_audiofile(filename, modus_operandi="repeat",
                                desired_length=desired_length, offset=steps*desired_length, pause=pause)
            mfcc, specbandwith, spectralcentroid, chroma, zerocrossrate, spectralroff = get_add_features(x, sr)
            feat_dict['mfcc'].append(mfcc)
            feat_dict['specbandwith'].append(specbandwith)
            feat_dict['spectralcentroid'].append(spectralcentroid)
            feat_dict['chroma'].append(chroma)
            feat_dict['zerocrossrate'].append(zerocrossrate)
            feat_dict['spectralroff'].append(spectralroff)  
            feat_dict['style'].append(style)
            feat_dict['bpm'].append(bpm)
                
    return feat_dict
    
    
def add_features_as_array_for_CNN(feat_dict_normalized):
    '''
    requires a dictionary with additional features already NORMALIZED (mfcc, specbandwith, spectralcentroid, 
            chroma, zerocrossrate, spectralroff) and not normalized (style = y) and bpm (no need to normalize?)
            
    returns a dictionary with keys: array (putting together all additional features in a single array:
                                             specbandwith, spectralcentroid,
                                            chroma, zeorcrossrate, spectralroff)
                                    style,
                                    bpm
                        each contains a list 
                        
    '''
    length = len(feat_dict['style'])
    cnn_add_feat_dict = {'feat_array':[],'stype':[],'bpm':[]}
    for i in range(length):
        style = feat_dict_normalized['style'][i]
        bpm = feat_dict_normalized['bpm'][i]
        feat_array = np.concatenate(
            feat_dict_normalized['norm_mfcc'][i],
            feat_dict_normalized['norm_specbandwith'][i],
            feat_dict_normalized['norm_spectralcentroid'][i],
            feat_dict_normalized['norm_chroma'][i],
            feat_dict_normalized['norm_zerocrossrate'][i],
            feat_dict_normalized['norm_spectralroff'][i],)
        cnn_add_feat_dict[feat_array].append(feat_array)
        cnn_add_feat_dict[style].append(style)
        cnn_add_feat_dict[bpm].append(bpm)
            
    return cnn_add_feat_dict

In [5]:
# db_audio = pd.read_csv(path + "info.csv")


def list_from_db(db_audio):
    list_of_samples = []
    for dataset in db_audio.iterrows():
        if dataset[1].beat_type=="beat":
            list_of_samples.append((dataset[1].duration, dataset[1].audio_filename, dataset[1].style, dataset[1].bpm))
    return list_of_samples

#list_from_db = list_from_db(db_audio)


def extracting_spectograms(list_from_db, desired_length, pause=1):
    preprocessed_data = []
    for audio_sample in list_from_db:
        filename = path + audio_sample[1]
        length = audio_sample[0]
        style = audio_sample[2]
        bpm = audio_sample[3]
        
        if length < desired_length: 
            x, sr = reading_audiofile(filename, modus_operandi="repeat",
                              desired_length=desired_length, offset=0.0, pause=pause)
            Xdb = spectogram(x, sr)
            preprocessed_data.append([Xdb, style, bpm])
            
        if length > desired_length:
            steps=int(length//desired_length)
            for i in range(steps):
                x, sr = reading_audiofile(filename, modus_operandi="no_change",
                            desired_length=desired_length, offset=i*desired_length)
                Xdb = spectogram(x, sr)
                preprocessed_data.append([Xdb, style, bpm])
                
            x, sr = reading_audiofile(filename, modus_operandi="repeat",
                                desired_length=desired_length, offset=steps*desired_length, pause=pause)
            Xdb = spectogram(x, sr)
            preprocessed_data.append([Xdb, style, bpm])
                
    return preprocessed_data

#prepoc=extracting_spectograms(list_from_db, desired_length=6, pause=1)

def reading_audiofile(filename, modus_operandi="no_change", desired_length=6, offset=0.0, pause=1):
    ''' reading in audiofiles
        requires filename (with path)
                modus_operandi:  "no_change" -> reading in audiofile in length, no padding
                                 "silence"-> padding smaller (than desired length) audio files with silence
                                 "repeat" -> padding smaller audio files with repetitive sound (+ pauses)
                desired_length:  in sec, if prespecified length is requested (default value 30)
                                 does not apply if "no_change" modus is chosen
                pause:           in sec, if modus "repeat" is chosen: if a pause between repetions is asked
        returns x (nd-array), sr (sample rate)
    '''

    if modus_operandi=="no_change":
         x , sr = librosa.load(filename, duration=desired_length, offset=0.0)

    if modus_operandi=="repeat":
        # == filling up smaller samples with repeated sounds ==
        x , sr = librosa.load(filename, duration= desired_length, offset=0.0)
        duration = x.shape[0]/sr        # in seconds
        if duration < desired_length:
            pause = int(pause * sr)     # seconds (first number) * sampling rate
            multiplier = int(desired_length // duration)
            single_x=np.append(np.zeros(pause),x)
            for i in range(multiplier):
                x = np.append(x, single_x)
            x=x[0:desired_length*sr]

    if modus_operandi=="silence":
        # == filling up smaller samples with silence ==
        x , sr = librosa.load(filename, duration= desired_length)
        duration = x.shape[0]/22000        # in seconds
        if duration < desired_length:
            filling_zeros = desired_length * sr - x.shape[0]
            x = np.append(x, np.zeros(filling_zeros))
            x=x[0:desired_length*sr]

    return x, sr

def spectogram(x, sr):
    ''' 
===  1 Spectogram  ===
 requires 1-dimensional array x(timeseries), sampling rate sr
 returns a vector of spectogram
    '''
    #performing short time- Fourier analysis
    X = librosa.stft(x)

    #amplitudes of given frequency at given time -> spectrogram
    Xdb = librosa.amplitude_to_db(abs(X))

    #plt.figure(figsize=(10, 10), dpi=50, frameon=False)
    #librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz') # for demonstration purposes
    #spect_audio = librosa.display.specshow(Xdb, sr=sr, x_axis=None, y_axis=None, cmap="Greys") # without axes
    return Xdb

In [6]:
def strip_genre(df):
    df.split()

In [7]:
path = '../raw_data/groove/'

In [8]:
db = pd.read_csv(path + "info.csv")
# db_audio = db_audio.iloc[0:10]
db.head()

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test


In [9]:
df_audio = db.drop(columns=['drummer', 'id', 'session', 'split', 'midi_filename'])
df_audio.head()

,style,bpm,beat_type,time_signature,audio_filename,duration
0,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308
1,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158
2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218
3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543
4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500


In [10]:
df_audio = df_audio[df_audio['beat_type'] == 'beat']
df_audio = df_audio.dropna()

In [11]:
df_audio['style'] = df_audio['style'].apply(lambda x: x.split('/')[0]) 
# df_audio.groupby(by='style_new').count()
# df_audio['style'] = df_audio['style_new']  

In [12]:
df_audio.head()

,style,bpm,beat_type,time_signature,audio_filename,duration
0,funk,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308
1,soul,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158
2,funk,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218
3,soul,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543
4,soul,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500


In [13]:
df_train, df_test = train_test_split(df_audio, test_size=0.2, random_state=7)

In [14]:
list_from_db = list_from_db(df_train)

In [15]:
len(list_from_db)

355

In [115]:
dictionary = extracting_add_features(list_from_db[0:355])

In [116]:
dictionary.keys()

dict_keys(['mfcc', 'specbandwith', 'spectralcentroid', 'chroma', 'zerocrossrate', 'spectralroff', 'style', 'bpm'])

In [117]:
len(dictionary['style'])

964

In [118]:
len(dictionary['specbandwith'])

964

In [119]:
dictionary['chroma'][0].shape

(12, 259)

In [120]:
dictionary['specbandwith'][0].shape

(1, 259)

In [121]:
df = pd.DataFrame.from_dict(dictionary)
df['chroma'][0].shape

(12, 259)

# Mean and standard deviation

In [122]:
df.head(1)

,mfcc,specbandwith,spectralcentroid,chroma,zerocrossrate,spectralroff,style,bpm
0,"[[-260.77158, -251.04285, -269.42813, -292.240...","[[3410.3093764508108, 3396.535848879602, 3230....","[[4974.285762405871, 5305.890006700972, 5686.2...","[[0.80164516, 0.48341665, 0.2846054, 0.3002181...","[[0.18359375, 0.2685546875, 0.36376953125, 0.4...","[[9011.6455078125, 9140.8447265625, 9259.27734...",jazz,125


## Simplified model with mean of means for 2-dimensional features

### Feature engineering

In [123]:
X = df.drop(columns=['mfcc', 'chroma', 'style', 'bpm'])

In [124]:
empty_df = pd.DataFrame()
for column in X.columns:
    empty_df[f'{column} mean'] = [X[column][i][0].mean() for i in range(len(X[column]))]
    empty_df[f'{column} std'] = [X[column][i][0].std() for i in range(len(X[column]))]
for column in ['mfcc', 'chroma']:
    empty_df[f'{column} mean'] = [df[column][j].mean(axis=1).mean() for j in range(len(df[column]))]
    empty_df[f'{column} std'] = [df[column][j].std(axis=1).mean() for j in range(len(df[column]))]

In [125]:
X = empty_df
y = df['style']

In [126]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tensorflow.keras.utils import to_categorical
labelencoder=LabelEncoder()
labelencoder.fit(y)
y_transformed = labelencoder.transform(y)


y_cat = to_categorical(y_transformed)
y_cat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

### Model Cross-validation

#### KNN Classifier

In [127]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate

knn_model = KNeighborsClassifier()
cv_results = cross_validate(knn_model, X, y_cat, cv=5, scoring=('accuracy', 'f1_samples'))

In [128]:
accuracy = cv_results['test_accuracy'].mean()
accuracy

0.23125

#### Random Forest

In [138]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
random_forest.fit(X, y_cat)

cv_forest = cross_validate(random_forest, X, y_cat, cv=5, scoring=('accuracy', 'f1_samples'))
forest_accuracy = cv_forest['test_accuracy'].mean()
forest_f1 = cv_forest['test_f1_samples'].mean()

print('accuracy =', forest_accuracy)
print('f1 =', forest_f1)

accuracy = 0.20118739205526767
f1 = 0.20118739205526767


#### Gradient Boosting Classifier

In [137]:
y_cat.shape

(964, 17)

In [169]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier


gradient_classifier = MultiOutputClassifier(GradientBoostingClassifier())
gradient_classifier.fit(X, y_cat)

cv_gradient = cross_validate(gradient_classifier, X, y_cat, cv=5, scoring=('accuracy', 'f1_samples'), error_score='raise')
gradient_accuracy = cv_gradient['test_accuracy'].mean()
gradient_f1 = cv_gradient['test_f1_samples'].mean()

print('accuracy =', gradient_accuracy)
print('f1 =', gradient_f1)


ValueError: y contains 1 class after sample_weight trimmed classes with zero weights, while a minimum of 2 classes are required.

In [ ]:
cv_gradient

#### XGBClassifier

In [168]:
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier


classifier = MultiOutputClassifier(XGBClassifier())

classifier.fit(X, y_cat)

cv_xgb = cross_validate(classifier, X, y_cat, cv=5, scoring=('accuracy', 'f1_samples'))
xgb_accuracy = cv_xgb['test_accuracy'].mean()
xgb_f1 = cv_xgb['test_f1_samples'].mean()

print('accuracy =', xgb_accuracy)
print('f1 =', xgb_f1)

[17:44:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/raphaelcommere/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:44:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

/Users/raphaelcommere/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:44:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

...

[17:45:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:45:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:45:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:45:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

/Users/raphaelcommere/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:45:03] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:45:03] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:45:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:45:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

/Users/raphaelcommere/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:45:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:45:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:45:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:45:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the o

## Model with means of columns for 2-dimensional features

In [350]:
df['mfcc'][1].mean(axis=1)

array([-5.2651624e+02, -2.8712662e+01,  1.4646410e+01, -1.3218553e+00,
       -1.2185350e+01, -3.7838848e+00, -1.6541931e+01, -3.2666261e+00,
       -1.1034554e+01, -4.9214196e+00, -1.2579289e+01,  4.2425504e+00,
       -8.1661615e+00,  8.9544859e+00,  1.3945830e+00,  1.8696419e+01,
       -2.9431400e-01,  2.7784011e+00, -7.7853932e+00,  6.2506118e+00],
      dtype=float32)

# All features + PCA

In [152]:
thisnewdf = pd.DataFrame(df['specbandwith'], columns= [f'specbandwith {n}' for n in range(len(df['specbandwith'][0][0]))])

In [153]:
for i in range(len(df['specbandwith'])):
    thisnewdf.loc[len(thisnewdf.index)] = df['specbandwith'][i][0].T

In [154]:
thisnewdf

,specbandwith 0,specbandwith 1,specbandwith 2,specbandwith 3,specbandwith 4,specbandwith 5,specbandwith 6,specbandwith 7,specbandwith 8,specbandwith 9,...,specbandwith 249,specbandwith 250,specbandwith 251,specbandwith 252,specbandwith 253,specbandwith 254,specbandwith 255,specbandwith 256,specbandwith 257,specbandwith 258
0,3410.309376,3396.535849,3230.951303,3235.318916,3232.160744,3112.911570,3112.040033,3075.445962,3123.479730,3242.512015,...,2730.336448,2725.959717,2723.265607,2671.408119,2752.352526,2838.689096,2874.953587,2871.652725,2816.372801,2763.828143
1,3410.309376,3396.535849,3230.951303,3235.318916,3232.160744,3112.911570,3112.040033,3075.445962,3123.479730,3242.512015,...,2730.336448,2725.959717,2723.265607,2671.408119,2752.352526,2838.689096,2874.953587,2871.652725,2816.372801,2763.828143
2,3410.309376,3396.535849,3230.951303,3235.318916,3232.160744,3112.911570,3112.040033,3075.445962,3123.479730,3242.512015,...,2730.336448,2725.959717,2723.265607,2671.408119,2752.352526,2838.689096,2874.953587,2871.652725,2816.372801,2763.828143
3,3747.892127,3698.503190,3735.398098,3722.224609,3644.081240,3386.318163,3385.648947,3626.638905,3632.692607,3598.308087,...,3690.100445,3683.396984,3543.141825,3275.030421,3230.964597,3428.241876,3253.688846,3307.315839,3446.804514,3534.697944
4,3747.892127,3698.503190,3735.398098,3722.224609,3644.081240,3386.318163,3385.648947,3626.638905,3632.692607,3598.308087,...,3690.100445,3683.396984,3543.141825,3275.030421,3230.964597,3428.241876,3253.688846,3307.315839,3446.804514,3534.697944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2886.074163,3099.988720,3220.652890,2981.120973,2927.851953,2759.233497,2587.194078,2478.673637,2826.696539,2923.789388
960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2886.074163,3099.988720,3220.652890,2981.120973,2927.851953,2759.233497,2587.194078,2478.673637,2826.696539,2923.789388
961,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2886.074163,3099.988720,3220.652890,2981.120973,2927.851953,2759.233497,2587.194078,2478.673637,2826.696539,2923.789388
962,2701.458496,3045.405703,3239.048678,3076.040406,3053.310621,2971.861005,2881.897364,2841.352256,2745.154344,2696.707514,...,2490.663916,2234.326501,2535.025008,2979.686856,2927.766806,2781.489943,2811.234127,2892.001240,2934.053880,2906.107931


In [219]:
specbandwith_coef_df = thisnewdf
specbandwith_coef_df

,specbandwith 0,specbandwith 1,specbandwith 2,specbandwith 3,specbandwith 4,specbandwith 5,specbandwith 6,specbandwith 7,specbandwith 8,specbandwith 9,...,specbandwith 249,specbandwith 250,specbandwith 251,specbandwith 252,specbandwith 253,specbandwith 254,specbandwith 255,specbandwith 256,specbandwith 257,specbandwith 258
0,3410.309376,3396.535849,3230.951303,3235.318916,3232.160744,3112.911570,3112.040033,3075.445962,3123.479730,3242.512015,...,2730.336448,2725.959717,2723.265607,2671.408119,2752.352526,2838.689096,2874.953587,2871.652725,2816.372801,2763.828143
1,3410.309376,3396.535849,3230.951303,3235.318916,3232.160744,3112.911570,3112.040033,3075.445962,3123.479730,3242.512015,...,2730.336448,2725.959717,2723.265607,2671.408119,2752.352526,2838.689096,2874.953587,2871.652725,2816.372801,2763.828143
2,3410.309376,3396.535849,3230.951303,3235.318916,3232.160744,3112.911570,3112.040033,3075.445962,3123.479730,3242.512015,...,2730.336448,2725.959717,2723.265607,2671.408119,2752.352526,2838.689096,2874.953587,2871.652725,2816.372801,2763.828143
3,3747.892127,3698.503190,3735.398098,3722.224609,3644.081240,3386.318163,3385.648947,3626.638905,3632.692607,3598.308087,...,3690.100445,3683.396984,3543.141825,3275.030421,3230.964597,3428.241876,3253.688846,3307.315839,3446.804514,3534.697944
4,3747.892127,3698.503190,3735.398098,3722.224609,3644.081240,3386.318163,3385.648947,3626.638905,3632.692607,3598.308087,...,3690.100445,3683.396984,3543.141825,3275.030421,3230.964597,3428.241876,3253.688846,3307.315839,3446.804514,3534.697944
5,3747.892127,3698.503190,3735.398098,3722.224609,3644.081240,3386.318163,3385.648947,3626.638905,3632.692607,3598.308087,...,3690.100445,3683.396984,3543.141825,3275.030421,3230.964597,3428.241876,3253.688846,3307.315839,3446.804514,3534.697944
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3205.393149,3062.308982,2974.633060,2927.527813,3029.562675,3115.199645,3133.012190,3161.817186,3074.887325,2883.114044
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3205.393149,3062.308982,2974.633060,2927.527813,3029.562675,3115.199645,3133.012190,3161.817186,3074.887325,2883.114044
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3205.393149,3062.308982,2974.633060,2927.527813,3029.562675,3115.199645,3133.012190,3161.817186,3074.887325,2883.114044
9,2976.464928,3046.121526,3244.706410,3661.358371,3747.467442,3606.603409,3523.939741,3372.064877,3158.089292,2725.902379,...,1895.262689,2123.924676,2664.550060,2958.771856,3241.901540,3121.380489,3006.907856,2952.401022,2870.539389,2841.056859


In [195]:
for i in range(len(df['specbandwith'])):
    thisnewdf[0]

TypeError: cannot concatenate object of type '<class 'numpy.float64'>'; only Series and DataFrame objs are valid

In [205]:
df['specbandwith'][0].T

array([[3410.30937645],
       [3396.53584888],
       [3230.95130333],
       [3235.31891633],
       [3232.16074353],
       [3112.91157048],
       [3112.04003298],
       [3075.44596153],
       [3123.47973008],
       [3242.51201524],
       [3276.02922508],
       [3338.00761011],
       [3269.2882385 ],
       [3434.41716142],
       [3584.53177812],
       [3563.50152406],
       [3536.39475718],
       [3527.56208025],
       [3501.3828902 ],
       [3553.02325969],
       [3145.95649015],
       [2916.13028437],
       [2840.13095032],
       [2796.67264552],
       [2802.25006402],
       [2816.18794895],
       [2834.77256585],
       [2885.03090759],
       [2849.77926467],
       [2807.46865253],
       [2859.42637859],
       [2922.23062662],
       [2948.61847777],
       [3058.5856056 ],
       [2876.76911303],
       [2788.6864587 ],
       [2730.95665527],
       [2711.03342006],
       [2706.03659187],
       [2750.64056871],
       [2775.82612069],
       [2781.685

In [149]:
range(len(df['specbandwith']))

range(0, 15)

In [172]:
thisnewdf

,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,256,257,258


In [100]:
thisnewdf[1]

Series([], Name: 1, dtype: object)

In [95]:
df['specbandwith'][0].T

array([[3410.30937645],
       [3396.53584888],
       [3230.95130333],
       [3235.31891633],
       [3232.16074353],
       [3112.91157048],
       [3112.04003298],
       [3075.44596153],
       [3123.47973008],
       [3242.51201524],
       [3276.02922508],
       [3338.00761011],
       [3269.2882385 ],
       [3434.41716142],
       [3584.53177812],
       [3563.50152406],
       [3536.39475718],
       [3527.56208025],
       [3501.3828902 ],
       [3553.02325969],
       [3145.95649015],
       [2916.13028437],
       [2840.13095032],
       [2796.67264552],
       [2802.25006402],
       [2816.18794895],
       [2834.77256585],
       [2885.03090759],
       [2849.77926467],
       [2807.46865253],
       [2859.42637859],
       [2922.23062662],
       [2948.61847777],
       [3058.5856056 ],
       [2876.76911303],
       [2788.6864587 ],
       [2730.95665527],
       [2711.03342006],
       [2706.03659187],
       [2750.64056871],
       [2775.82612069],
       [2781.685

In [23]:
X = df.drop(columns=['style'])
y = df['style']

In [27]:
df.columns

Index(['mfcc', 'specbandwith', 'spectralcentroid', 'chroma', 'zerocrossrate',
       'spectralroff', 'style', 'bpm'],
      dtype='object')

In [44]:
len(d['specbandwith'])

15